# BLEND052
- Add keras model

In [8]:
import pandas as pd
sch_oof = pd.read_csv('../oof/SchNet_ensemble_v5_oof.csv')
sch_pred = pd.read_csv('../submissions/SchNet_ensemble_v5.csv')

In [9]:
sch_oof = sch_oof.rename(columns={'scalar_coupling_constant':'SCH'})
sch_pred = sch_pred.rename(columns={'scalar_coupling_constant':'SCH'})

In [10]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

def log_mean_absolute_error(y_true, y_pred):
    return np.log(mean_absolute_error(y_true, y_pred))
%matplotlib inline

In [12]:
types = ['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']

def get_sub_oof(t):
    """
    Grab the sub and oof files for a give type. 
    """
    oof = []
    sub = []
    model_ids = []
    for file in sorted(os.listdir(f'../type_results/{t}/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            df.index = df.id.values
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/{file}')
            df.index = df.id.values
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']
    for file in sorted(os.listdir(f'../type_results/{t}/old/')):
        if ('sub' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            model_ids.append(model_id)
            df = pd.read_parquet(f'../type_results/{t}/old/{file}')
            df.index = df.id.values
            if len(sub) == 0:
                sub = df.rename(columns={'scalar_coupling_constant': model_id})
            else:
                sub[model_id] = df['scalar_coupling_constant']
        elif ('oof' in file) and ('3folds' in file):
            model_id = file[:4] + file.split('MAE_')[1].replace('L','')
            #print(model_id)
            df = pd.read_parquet(f'../type_results/{t}/old/{file}')
            df.index = df.id.values
            if len(oof) == 0:
                oof = df.rename(columns={'oof_preds': model_id})
            else:
                oof[model_id] = df['oof_preds']

    sub = sub.loc[sub['type'] == t]
    oof = oof.loc[oof['type'] == t]
    return sub, oof, model_ids

def get_best_weights(oof, l5, t):
    if len(l5) == 1:
        return [1]
    lmae_best = 0
    for x in tqdm(range(0, 5000)):
        ws = []
        for x in range(0, len(l5)):
            if x == len(l5)-1:
                ws.append(random.randint(1, 50))
            else:
                ws.append(random.randint(0, 50))
        idx = 0
        blend = np.zeros(len(oof[l5[0]]))
        for w in ws:
            blend += w * oof[l5[idx]]
            idx += 1
        blend = blend / np.sum(ws)
        #print(blend.head())
        lmae = log_mean_absolute_error(oof['scalar_coupling_constant'],
                            blend)
        if lmae < lmae_best:
            lmae_best = lmae
            bws = ws
    print(f'{t}: ===== Best score {lmae_best:0.5f} - with weights {bws}')
    return bws # Return the best weights

def create_best_blends():
    oofs = []
    subs = []
    for t in types:
        print(f'{t}: Running for type {t}')
        tsub, toof, tmodel_ids = get_sub_oof(t)
        toof = toof.merge(sch_oof, on='id')
        tsub = tsub.merge(sch_pred, on='id')
        tmodel_ids = tmodel_ids + ['SCH']
        print(f'{t}: Has model names {tmodel_ids}')
        #l5 = tmodel_ids[-5:] # Last 5 models
        bws = get_best_weights(toof, tmodel_ids, t)
        tot_weight = np.sum(bws)
        tsub['scalar_coupling_constant'] = 0
        toof['oof_blend'] = 0
        idx = 0
        for w in bws:
            tsub['scalar_coupling_constant'] += (w * tsub[tmodel_ids[idx]])
            toof['oof_blend'] += (w * toof[tmodel_ids[idx]])
            idx += 1
        tsub['scalar_coupling_constant'] /= tot_weight
        toof['oof_blend'] /= tot_weight
        tsub = tsub[['id','type','scalar_coupling_constant']].copy()
        toof = toof[['id','type','scalar_coupling_constant','oof_blend']].copy()
        subs.append(tsub)
        oofs.append(toof)
    final_oof = pd.concat(oofs).sort_values('id').reset_index(drop=True)
    final_sub = pd.concat(subs).sort_values('id').reset_index(drop=True)
    return final_oof, final_sub

final_oof, final_sub = create_best_blends()
print('Done')

1JHC: Running for type 1JHC


  0%|          | 20/5000 [00:00<00:24, 199.35it/s]

1JHC: Has model names ['K004-0.7783', 'M048-0.8258', 'M053-0.8639', 'M055-0.8641', 'M058-0.8972', 'M059-0.9326', 'M061-0.9254', 'M047-0.7977', 'M049-0.791', 'SCH']


100%|██████████| 5000/5000 [00:29<00:00, 167.46it/s]


1JHC: ===== Best score -1.25685 - with weights [11, 16, 7, 10, 4, 23, 3, 4, 14, 45]
1JHN: Running for type 1JHN


...

3JHH: ===== Best score -2.59327 - with weights [14, 12, 2, 6, 22, 7, 14, 2, 42]
3JHN: Running for type 3JHN


  0%|          | 22/5000 [00:00<00:23, 210.15it/s]

3JHN: Has model names ['K004-2.284', 'M050-2.383', 'M052-2.39', 'M056-2.4', 'M501-2.416', 'M047-2.356', 'M048-2.321', 'M054-2.277', 'M055-2.363', 'SCH']


100%|██████████| 5000/5000 [00:25<00:00, 193.43it/s]


3JHN: ===== Best score -2.75678 - with weights [0, 16, 0, 0, 3, 15, 12, 0, 27, 36]
Done


## Score

In [13]:
for i, d in final_oof.groupby('type'):
    score = mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend'])
    lscore = np.log(mean_absolute_error(d['scalar_coupling_constant'], d['oof_blend']))
    print(i,'\t {:0.5f} \t {:0.5f}'.format(score, lscore))
glmae = group_mean_log_mae(final_oof['scalar_coupling_constant'], final_oof['oof_blend'], final_oof['type'])
print('\nGroup LMAE')
print('{:0.5f}'.format(glmae))

1JHC 	 0.28455 	 -1.25685
1JHN 	 0.15167 	 -1.88604
2JHC 	 0.10593 	 -2.24498
2JHH 	 0.06667 	 -2.70806
2JHN 	 0.07695 	 -2.56454
3JHC 	 0.12241 	 -2.10036
3JHH 	 0.07478 	 -2.59327
3JHN 	 0.06350 	 -2.75678

Group LMAE
-2.26386


# Validate Sub

In [8]:
sub_good = pd.read_csv('../submissions/BLEND044_sub_-2.03355CV.csv')
final_sub['good_scc'] = sub_good['scalar_coupling_constant']
final_sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.0,1.0
good_scc,1.0,1.0


In [9]:
print('MAE vs Good Sub')
for i, d in final_sub.groupby('type'):
    mae_t = mean_absolute_error(d['good_scc'], d['scalar_coupling_constant'])
    corr_t = np.corrcoef(d['good_scc'], d['scalar_coupling_constant'])[1][0]
    print(f'{i} - {mae_t:0.5f} - {corr_t:0.5f}')

MAE vs Good Sub
1JHC - 0.00886 - 1.00000
1JHN - 0.02600 - 0.99999
2JHC - 0.00899 - 0.99999
2JHH - 0.01232 - 0.99999
2JHN - 0.00865 - 0.99999
3JHC - 0.00654 - 0.99999
3JHH - 0.00997 - 0.99999
3JHN - 0.00476 - 0.99998


# Create Submission and OOF Files

In [10]:
import json
import os.path
import re
import ipykernel
import requests

#try:  # Python 3
#    from urllib.parse import urljoin
#except ImportError:  # Python 2
#    from urlparse import urljoin

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name():
    """
    Return the full path of the jupyter notebook.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [12]:
BLEND_NUMBER = get_notebook_name().split('/')[-1].replace('.ipynb','').replace('-','')
print(f'NAME TO SAVE {BLEND_NUMBER}')

NAME TO SAVE BLEND051


In [13]:
# Save the Results
final_sub[['id','scalar_coupling_constant']].to_csv(f'../submissions/{BLEND_NUMBER}_sub_{glmae:0.5f}CV.csv', index=False)
final_oof.to_csv(f'../oof/{BLEND_NUMBER}_oof_{glmae:0.5f}CV.csv', index=False)